In [41]:
CSV_IN = 'datos/Base datos_INTA_ascii_limpio.csv'

In [42]:
print 'Cargando datos de "%s"' % CSV_IN
df = pd.read_csv(CSV_IN)
print 'Cargado dataframe de tamaño %d x %d' % df.shape

Cargando datos de "datos/Base datos_INTA_ascii_limpio.csv"
Cargado dataframe de tamaño 423 x 35


In [43]:
# Estos son funciones para hacer cuentas claras y minimalistas

def truncate_str(s, n=15):
    s = str(s)
    if len(s) > n:
        return s[:(n-3)] + '...'
    else:
        return s

def smart_value_counts(s, top=10, normalize=True, percent=False):
    vc = s.value_counts(ascending=False, normalize=normalize)
    vc.index = [truncate_str(i) for i in vc.index]
    out = vc[:top]
    if vc.sum() > out.sum():
        other = pd.Series([vc.sum() - out.sum()], index=['[OTROS] (%d tipos)' % (len(vc) - len(out))])
        out = out.append(other)
    na = 1.0 - vc.sum()
    out = out.append(pd.Series([na], index=['[NA (%.0f%%)]' % (na * 100)]))
    if percent:
        return out.apply(lambda p: '%.2f%%' % (p*100))
    return out

smart_value_counts(df['campana'], top=5, normalize=True, percent=True)

2011                 17.49%
2012                 14.89%
2014                 13.00%
2009                 12.77%
2008                 12.53%
[OTROS] (3 tipos)    29.31%
[NA (0%)]             0.00%
dtype: object

In [44]:
# df.describe()

In [45]:
smart_value_counts(df['campana'])

2011         0.174941
2012         0.148936
2014         0.130024
2009         0.127660
2008         0.125296
2010         0.101655
2006         0.101655
2007         0.089835
[NA (0%)]    0.000000
dtype: float64

In [46]:
try:
    os.mkdir('figuras/explorar/')
except OSError:
    pass

def figura_hist(s, fig, ax, figname):
    s = s.astype(float)
    s.hist(ax=ax, color=COLOR_HIST, weights=np.zeros(s.count()) + 1. / len(s),
           alpha=BAR_ALPHA)
    ax.set_yticklabels(['%.0f%%' % (t*100) for t in ax.get_yticks()])
    ax.set_title(c)
    na = 1.0 - float(s.count()) / len(s)
    ax.plot([min(ax.get_xticks()), max(ax.get_xticks())], [na, na],
            color=COLOR_NA, label='NA (%.0f%%)' % (na * 100), alpha=0.8,
            linestyle='--')
    ax.legend(loc='upper right')
    if RUGPLOT:
        sns.rugplot(s, color=COLOR_HIST, alpha=1.0, ax=ax)
    fig.savefig('figuras/explorar/hist_%s.png' % figname)

def figura_bar(s, fig, ax, figname):
    s = smart_value_counts(s)
    s[::-1].plot(ax=ax, kind='barh', color=[COLOR_NA] +
                 [COLOR_BAR]*(len(s) - 1), alpha=BAR_ALPHA)
    ax.set_xticklabels(['%.0f%%' % (t*100) for t in ax.get_xticks()])
    fig.savefig('figuras/explorar/bar_%s.png' % figname)

In [47]:
# Creamos una figura por cada columno, sumarizandolo visualmente

# Configuraction visual
COLOR_HIST = sns.color_palette()[0]
COLOR_BAR = sns.color_palette()[1]
COLOR_NA = (.8, .2, .2)
BAR_ALPHA = 0.8
RUGPLOT = False

saltados, numericos = 0, 0
for i, c in enumerate(sorted(df.columns)):
    if df[c].count() == 0:
        print '**SALTANDO "%s"** (todos NA)' % c
        saltados += 1
        continue
    print '[%d/%d] %s' % (i+1, len(df.columns), c),
    
    fig, ax = plt.subplots(1)
    fig.subplots_adjust(left=0.2)
    ax.set_title(c)
    s = df[c]
    figname = ''.join([ch for ch in c.lower().replace(' ', '_')
                       if ch.isalnum() or ch in ('_-')])
    try:
        figura_hist(s, fig, ax, figname)
        numericos += 1
        print '(float)'
    except ValueError:
        figura_bar(s, fig, ax, figname)
        print '(categorico)'
    plt.close()

print 'Columnos: [%d numericos, %d categoricos, %d saltados (vacios), %d total]' % (
    numericos, len(df.columns) - numericos - saltados, saltados, len(df.columns))

[1/35] analisis_suelo_k (float)
[2/35] analisis_suelo_n (float)
[3/35] analisis_suelo_p (float)
[4/35] campana (float)
[5/35] cultivo_antecesor (categorico)
[6/35] densidad_siembra_kg_ha (float)
[7/35] factor_rend_densidad_pl_m2 (categorico)
[8/35] factor_rend_pctfactor (float)
[9/35] factor_rend_pcthcosecha (float)
[10/35] fenologia_cosecha_fecha (categorico)
[11/35] fenologia_dpf_fecha (categorico)
[12/35] fenologia_emerg_cosecha_dias (float)
[13/35] fenologia_emerg_dpf_dias (float)
[14/35] fenologia_emerg_flor_50pct_dias (float)
[15/35] fenologia_emergencia_70pct_fecha (categorico)
[16/35] fenologia_flor_50pct_fecha (categorico)
[17/35] fenologia_siembra_fecha (categorico)
[18/35] fert_cober_clk_cant_kg_ha (float)
[19/35] fert_cober_dpf_cant_n_kg_ha (float)
[20/35] fert_cober_dpf_fecha (categorico)
[21/35] fert_cober_preriego_cant_n_kg_ha (float)
[22/35] fert_cober_preriego_fecha (categorico)
[23/35] fert_cober_preriego_formulado (categorico)
[24/35] id (categorico)
[25/35] rendimie

In [48]:
clima = pd.read_csv('datos/clima_datos_unidos.csv', index_col='Date')
for c in clima.columns:
    figname = 'clima_' + c.lower()
    print figname
    fig, ax = plt.subplots(1)
    fig.subplots_adjust(left=0.2)
    ax.set_title(c)
    s = clima[c]
    figura_hist(s, fig, ax, figname)
    plt.close()

clima_esol
clima_rain
clima_rhum
clima_tmax
clima_tmin
